# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8a28986070>
├── 'a' --> tensor([[-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993]])
└── 'x' --> <FastTreeValue 0x7f8a289cbfd0>
    └── 'c' --> tensor([[ 0.8956,  0.5360,  2.2111, -1.9051],
                        [ 0.1788,  0.3119, -1.1663, -1.5693],
                        [-0.8482,  0.1421,  0.8436,  1.3406]])

In [4]:
t.a

tensor([[-0.7833, -0.1307,  0.9025],
        [ 1.3312,  0.7289, -0.4993]])

In [5]:
%timeit t.a

62.3 ns ± 0.0455 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8a28986070>
├── 'a' --> tensor([[-0.0773, -0.0061,  0.2347],
│                   [ 2.1868,  0.4502,  0.4582]])
└── 'x' --> <FastTreeValue 0x7f8a289cbfd0>
    └── 'c' --> tensor([[ 0.8956,  0.5360,  2.2111, -1.9051],
                        [ 0.1788,  0.3119, -1.1663, -1.5693],
                        [-0.8482,  0.1421,  0.8436,  1.3406]])

In [7]:
%timeit t.a = new_value

62.6 ns ± 0.00946 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7833, -0.1307,  0.9025],
               [ 1.3312,  0.7289, -0.4993]]),
    x: Batch(
           c: tensor([[ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406]]),
       ),
)

In [10]:
b.a

tensor([[-0.7833, -0.1307,  0.9025],
        [ 1.3312,  0.7289, -0.4993]])

In [11]:
%timeit b.a

56.6 ns ± 0.0277 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8408,  0.2969,  0.4565],
               [ 0.8201,  0.2900, -1.7988]]),
    x: Batch(
           c: tensor([[ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

848 ns ± 0.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

133 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 389 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f89852881f0>
├── 'a' --> tensor([[[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]],
│           
│                   [[-0.7833, -0.1307,  0.9025],
│                    [ 1.3312,  0.7289, -0.4993]]])
└── 'x' --> <FastTreeValue 0x7f897e6951f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8a289cb040>
├── 'a' --> tensor([[-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993],
│                   [-0.7833, -0.1307,  0.9025],
│                   [ 1.3312,  0.7289, -0.4993]])
└── 'x' --> <FastTreeValue 0x7f89fbeda1c0>
    └── 'c' --> tensor([[ 0.8956,  0.5360,  2.2111, -1.9051],
                        [ 0.1788,  0.3119, -1.1663, -1.5693],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 77.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.8956,  0.5360,  2.2111, -1.9051],
                       [ 0.1788,  0.3119, -1.1663, -1.5693],
                       [-0.8482,  0.1421,  0.8436,  1.3406]],
              
                      [[ 0.8956,  0.5360,  2.2111, -1.9051],
                       [ 0.1788,  0.3119, -1.1663, -1.5693],
                       [-0.8482,  0.1421,  0.8436,  1.3406]],
              
                      [[ 0.8956,  0.5360,  2.2111, -1.9051],
                       [ 0.1788,  0.3119, -1.1663, -1.5693],
                       [-0.8482,  0.1421,  0.8436,  1.3406]],
              
                      [[ 0.8956,  0.5360,  2.2111, -1.9051],
                       [ 0.1788,  0.3119, -1.1663, -1.5693],
                       [-0.8482,  0.1421,  0.8436,  1.3406]],
              
                      [[ 0.8956,  0.5360,  2.2111, -1.9051],
                       [ 0.1788,  0.3119, -1.1663, -1.5693],
                       [-0.8482,  0.1421,  0.8436,  1.3406]],

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406],
                      [ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406],
                      [ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406],
                      [ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406],
                      [ 0.8956,  0.5360,  2.2111, -1.9051],
                      [ 0.1788,  0.3119, -1.1663, -1.5693],
                      [-0.8482,  0.1421,  0.8436,  1.3406],
                      [ 0.8956,  0.5360,  2.2111, -1.9051],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 354 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

302 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
